In [0]:
#for google-colab
!pip install python-mnist
!wget https://github.com/sorki/python-mnist/blob/master/bin/mnist_get_data.sh
!mnist_get_data.sh

--2020-06-10 09:21:29--  https://github.com/sorki/python-mnist/blob/master/bin/mnist_get_data.sh
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘mnist_get_data.sh’

mnist_get_data.sh       [ <=>                ]  70.58K   413KB/s    in 0.2s    

2020-06-10 09:21:30 (413 KB/s) - ‘mnist_get_data.sh’ saved [72279]

--2020-06-10 09:21:33--  http://yann.lecun.com/exdb/mnist/
Resolving yann.lecun.com (yann.lecun.com)... 104.28.6.204, 172.67.171.76, 104.28.7.204, ...
Connecting to yann.lecun.com (yann.lecun.com)|104.28.6.204|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/index.html.tmp’

index.html.tmp          [ <=>                ]  28.49K  --.-KB/s    in 0s      

2020-06-10 09:21:33 (376 MB/s) - ‘data/index.html.tmp’ saved [29175]

Loading robots.txt; please igno

In [0]:
from mnist import MNIST
import numpy as np
from scipy.spatial.distance import cdist
import math
#https://pypi.org/project/python-mnist
mndata = MNIST('./data/')
images, labels = mndata.load_training()
%matplotlib inline
import matplotlib.pyplot as plt
import copy
from numpy import linalg
import time

l = len(images)
n = len(images[0])
m = int(np.sqrt(n))

def mnist(eps, p, q):
    p, q = np.float64(images[p]), np.float64(images[q])
    p, q = p / sum(p), q / sum(q)
    
    p = (1-eps/8)*p + eps/(8*n)
    q = (1-eps/8)*q + eps/(8*n)
    
    return p, q

def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

C = np.arange(m)
C = cartesian_product(C, C)
C = cdist(C, C)
C /= np.max(C)
C.max()

1.0

In [0]:
one = np.ones(n, np.float64)
I = np.ones([n,n], np.float64)
def f_true(x):
    return (C * x).sum()

def B_round(x):
    r = p_ref / x.dot(one)
    r[r>1] = 1.
    F = np.diag(r).dot(x)
    c = q_ref / (x.T).dot(one)
    c[c>1] = 1.
    F = F.dot(np.diag(c))
    err_r = p_ref - F.dot(one)
    err_c = q_ref - (F.T).dot(one)
    return F + np.outer(err_r, err_c) / abs(err_r).sum()

In [0]:
def phi_(gamma, lamu):
    A = (-C/gamma + np.outer(lamu[:n], one) + np.outer(one, lamu[n:]))
    a = A.max()
    A-=a
    s = a+np.log(np.exp(A).sum())
    return gamma*(-lamu[:n].dot(p) - lamu[n:].dot(q) + s)

def f_(gamma, x):
    y = (x.reshape(-1)).copy()
    y[x.reshape(-1) == 0.] = 1.
    y = y.reshape(n, -1)
    return (C * x).sum() + gamma * (x * np.log(y)).sum()

In [0]:
import warnings
def AGMsDR(x_start):
    f = phi_
    f_primal = lambda x: f_(gamma, x)

    mu=0 #ONLY!
    A = 0.
    tau = 1.
    x, v = x_start.copy(), x_start.copy()
    primal_var = np.zeros_like(K)
    
    start_time = time.perf_counter()
    f_x = None
    t=np.ones(2, np.float64)
    i=0
    while True:
        y, f_y, grad_f_y, norm2_grad_f_y, x, f_x, B, t[i%2] = agmsdr_iter(i, t[i%2], f_x, x, v)
        #print('\n')
        g = (f_x - f_y)
        a = norm2_grad_f_y + 2*mu*g
        b = 2*mu*A*g + 2*tau*g - mu*tau*(((v - y)*(v - y)).sum())
        c = 2*A*tau*g

        alpha = (-b + np.sqrt(b*b - 4*a*c)) / 2 / a
        
        primal_var = alpha * B + primal_var*A
        A = A + alpha
        
        v = tau*v + mu*alpha*y - alpha * (grad_f_y)
        tau = tau+mu*alpha
        v/=tau
        
        primal_var/=A
        
        #print((C * (B_round(primal_var) - primal_var)).sum(), f_primal(primal_var) + f_x, eps/6)
        if (C * (B_round(primal_var) - primal_var)).sum() <= eps/6 and abs(f_primal(primal_var) + f_x) <= eps/6:
            return time.perf_counter() - start_time
        i-=-1
            
    return np.array(history_f), np.array(history_time)

def agmsdr_iter(i, t, f_x, x, v):
    def check(t, forcereturn=False):
        
        #print(i,': ', t)
        y = v + t * (x-v)
        logB = (K + np.outer(y[:n], one) + np.outer(one, y[n:]))
        max_logB = (logB).max()
        logB_stable = logB - max_logB
        B_stable = np.exp(logB_stable)
        u_hat_stable, v_hat_stable = B_stable.dot(one), B_stable.T.dot(one)
        
        Bs_stable = u_hat_stable.sum()

        f_y = gamma*(-y[:n].dot(p) - y[n:].dot(q) + np.log(Bs_stable) + max_logB)
        grad_f_y = gamma*np.concatenate((-p + u_hat_stable/Bs_stable, -q + v_hat_stable/Bs_stable),0)
        if (grad_f_y.dot(v-y) >= 0 and f_x >= f_y) or forcereturn:
            gu, gv = (grad_f_y[:n]**2).sum(), (grad_f_y[n:]**2).sum()
            norm2_grad_f_y = (gu+gv)

            x_new = y.copy()    
            if gu > gv:
                with warnings.catch_warnings():
                    warnings.filterwarnings('error')
                    try:
                        ustep = p/u_hat_stable
                    except Warning as e:
                        u_hat_stable/=u_hat_stable.max()
                        u_hat_stable[u_hat_stable<1e-150] = 1e-150
                        ustep = p/u_hat_stable
                        #print('catchu')
                    
                
                ustep/=ustep.max()
                x_new[:n]+=np.log(ustep)
                Z=ustep[:,None]*B_stable
            else:
                with warnings.catch_warnings():
                    warnings.filterwarnings('error')
                    try:
                        vstep = q/v_hat_stable
                    except Warning as e:
                        v_hat_stable/=v_hat_stable.max()
                        v_hat_stable[v_hat_stable<1e-150] = 1e-150
                        vstep = q/v_hat_stable
                        #print('catchv')

                vstep/=vstep.max()
                x_new[n:]+=np.log(vstep)
                Z=B_stable*vstep[None,:]
            f_x_new=gamma*(np.log(Z.sum())+max_logB-x_new[:n].dot(p)-x_new[n:].dot(q))
            #print(phi_(gamma,x_new) -f_x_new)

            return True, (y, f_y, grad_f_y , norm2_grad_f_y, x_new, f_x_new, B_stable/Bs_stable, t) #f(x_new) can be optimized
        else:
            return False, grad_f_y

    if f_x==None: f_x = phi_(gamma, x) 
    tl=None
    tr=np.float128(t)
    k=0
    while True: #find right endpoint for line search
        is_ok, ret = check(tr)
        if is_ok:
            return ret
        else:
            gr = ret
        if gr.dot(x-v) < 0:
            tl=tr
            tr= 1 + 1e-8 * k**10
        else:
            break
        k-=-1

    
    # small step to the left    
    tmp=max(0, min(tr-(1 - (i+1)/(i+2)), (i+1)/(i+2))) 
    while tl==None: #find left endpoint for line search
        is_ok, ret = check(tmp)
        if is_ok:
            return ret
        else:
            gtmp = ret
        if gtmp.dot(x-v) <= 0:
            tl=tmp
            break
        else:
            tr=tmp
        tmp = 1 - (1-tmp**(4)) 

    
    # search in [tl,tr]
    while True:
        if tr < 0.8 or tl >= 1.:
            tc = tl + (tr-tl)*3/5
        else:
            tc = tl + (tr-tl)*4/5

        is_ok, ret = check(tc, tc==tr or tc==tl)

        if is_ok:
            return ret
        else:
            gc = ret
        if gc.dot(x-v) > 0:
            tr=tc
        else:
            tl=tc

In [0]:
def aam(x0, min_iter=0, min_time=0):
    L=1
    step = 2
    xi = np.zeros_like(x0)
    eta = xi.copy()
    zeta = xi.copy()
    eta_new = xi.copy()
    zeta_new = xi.copy()
    grad2 = alpha_new = alpha = 0
    ustep = np.zeros_like(x0[:n])
    vstep = np.zeros_like(ustep)
    
    f = lambda x: phi_(gamma, x)
    f_primal = lambda x: f_(gamma, x)
    
    K=-C/gamma
    primal_var = np.zeros_like(K)
    
    start_time = time.perf_counter()
    while True:
        L_new = L/step
        while True:
            alpha_new = 1/2/L_new + np.sqrt(1/4/L_new/L_new + alpha*alpha*L/L_new)
            tau = 1/alpha_new/L_new
            xi = tau*zeta + (1-tau)*eta
            
            ##############
            logB = (K + np.outer(xi[:n], one) + np.outer(one, xi[n:]))
            max_logB =logB.max()
            logB_stable = logB - max_logB

            B_stable = np.exp(logB_stable)
            u_hat_stable, v_hat_stable = B_stable.dot(one), B_stable.T.dot(one)
            
            Bs_stable = u_hat_stable.sum()

            f_xi = gamma*(-xi[:n].dot(p) - xi[n:].dot(q) + np.log(Bs_stable) + max_logB)
            grad_f_xi = gamma*np.concatenate((-p + u_hat_stable/Bs_stable, -q + v_hat_stable/Bs_stable),0)
            
            gu, gv = (grad_f_xi[:n]**2).sum(), (grad_f_xi[n:]**2).sum()
            norm2_grad_f_xi = (gu+gv)

            if gu > gv:
                with warnings.catch_warnings():
                    warnings.filterwarnings('error')
                    try:
                        ustep = p/u_hat_stable
                    except Warning as e:
                        u_hat_stable/=u_hat_stable.max()
                        u_hat_stable[u_hat_stable<1e-150] = 1e-150
                        ustep = p/u_hat_stable
                        #print('catchu')
                    
                
                ustep/=ustep.max()
                xi[:n]+=np.log(ustep)
                Z=ustep[:,None]*B_stable
            else:
                with warnings.catch_warnings():
                    warnings.filterwarnings('error')
                    try:
                        vstep = q/v_hat_stable
                    except Warning as e:
                        v_hat_stable/=v_hat_stable.max()
                        v_hat_stable[v_hat_stable<1e-150] = 1e-150
                        vstep = q/v_hat_stable
                        #print('catchv')

                vstep/=vstep.max()
                xi[n:]+=np.log(vstep)
                Z=B_stable*vstep[None,:]
            ##############
            f_eta_new=gamma*(np.log(Z.sum())+max_logB-xi[:n].dot(p)-xi[n:].dot(q))
            
            #print(L_new)
            if f_eta_new <= f_xi - (norm2_grad_f_xi)/2/L_new: # can be optimized 2 itmes
                primal_var = (alpha_new * B_stable/Bs_stable + L * alpha**2 * primal_var) /(L_new*alpha_new**2)
                
                zeta -= alpha_new * grad_f_xi
                #eta = eta_new.copy()
                eta = xi.copy()
                alpha = alpha_new
                L = L_new
                
                break
            L_new*=step
        
        #print((C * (B_round(primal_var) - primal_var)).sum(), f_primal(primal_var) + f_eta_new, eps/6)
        if (C * (B_round(primal_var) - primal_var)).sum() <= eps/6 and abs(f_primal(primal_var) + f_eta_new) <= eps/6:
            return time.perf_counter() - start_time
    return np.array(history_f), np.array(history_time)


In [0]:
def apd(x0):
    L = 1.
    betta = 0.
    alpha = 0.
    primal_var = 0.*np.ones([n, n], np.float64)
    z = np.zeros(2*n, np.float64)
    z_new = np.zeros(2*n, np.float64)
    eta = np.zeros(2*n, np.float64)
    eta_new  = np.zeros(2*n, np.float64)
    grad_phi_new = np.zeros(2*n, np.float64)
    
    f = lambda x: phi_(gamma, x)
    f_primal = lambda x: f_(gamma, x)
    
    k=0
    start_time = time.perf_counter()
    while True:
        L = L / 2
        while True:
            alpha_new = (1 + np.sqrt(4*L*betta + 1)) / 2 / L
            betta_new = betta + alpha_new
            tau = alpha_new / betta_new
            lamu_new = tau * z + (1 - tau) * eta
                    
            logB = (K + np.outer(lamu_new[:n], one) + np.outer(one, lamu_new[n:]))
            max_logB =logB.max()
            logB_stable = logB - max_logB

            B_stable = np.exp(logB_stable)
            u_hat_stable, v_hat_stable = B_stable.dot(one), B_stable.T.dot(one)
            
            Bs_stable = u_hat_stable.sum()

            phi_new = gamma*(-lamu_new[:n].dot(p) - lamu_new[n:].dot(q) + np.log(Bs_stable) + max_logB)
            grad_phi_new = gamma*np.concatenate((-p + u_hat_stable/Bs_stable, -q + v_hat_stable/Bs_stable),0)

            
            z_new = z - alpha_new * grad_phi_new
            eta_new = tau * z_new + (1-tau) * eta

            phi_eta = f(eta_new)

            if phi_eta <= phi_new + grad_phi_new.dot(eta_new - lamu_new) + L * ((eta_new - lamu_new)**2).sum() / 2:
                alpha, betta = alpha_new, betta_new
                z = z_new.copy()
                eta = eta_new.copy()
                break    
            L = L * 2
        primal_var = tau * B_stable/Bs_stable + (1 - tau) * primal_var
        
        k=k+1
        if (C * (B_round(primal_var) - primal_var)).sum() <= eps/6 and abs(f_primal(primal_var) + phi_eta) <= eps/6:
            return time.perf_counter() - start_time

In [0]:
def AARBCD(x0):
    f_primal = lambda x: f_(gamma, x)
    sigma=1
    X = np.zeros_like(C); #output primal variable
    primal_var = np.zeros_like(C); #output primal variable
    psi_y=0;
    psi_x = 0; #objective values at points x_k which are accumulated in the model of the function
    grad_psi_x = np.zeros_like(x0); #gradient values at x_k which are accumulated in the model of the function
    psi_y = 0; #objective values at points y_k for which usually the convergence rate is proved
    gap = 0;
    a=1;
    A = 0; # init array of A_k
    L = 0; #init array of L_k
    x = np.zeros_like(x0)
    y = np.zeros_like(x0)
    z = np.zeros_like(x0)
    a = 1; #set L_0

    k=0
    start_time = time.perf_counter()
    while True:
        flag = 1; #flag of the end of the inner cycle
        j = 0; #corresponds to j_k
        while flag > 0:
            a_t = 2**(1-1*j)*a; #current trial for a
            A_t = A + a_t; #trial of A_k
            tau = a_t / A_t; #trial of \tau_{k}
            
            x_t = tau * z + (1 - tau) * y; #trial for x_k ??? ??? ? ???????

            logB = (K + np.outer(x_t[:n], one) + np.outer(one, x_t[n:]))
            max_logB =logB.max()
            logB_stable = logB - max_logB

            B_stable = np.exp(logB_stable)
            u_hat_stable, v_hat_stable = B_stable.dot(one), B_stable.T.dot(one)
            
            Bs_stable = u_hat_stable.sum()

            grad_f_xi = gamma*np.concatenate((-p + u_hat_stable/Bs_stable, -q + v_hat_stable/Bs_stable),0)
            gu, gv = (grad_f_xi[:n]**2).sum(), (grad_f_xi[n:]**2).sum()
            if k%2==1:#gu > gv:
                with warnings.catch_warnings():
                    warnings.filterwarnings('error')
                    try:
                        ustep = p/u_hat_stable
                    except Warning as e:
                        u_hat_stable/=u_hat_stable.max()
                        u_hat_stable[u_hat_stable<1e-150] = 1e-150
                        ustep = p/u_hat_stable
                    
                
                ustep/=ustep.max()
                #print('ueq', np.array_equal(np.diag(ustep).dot(B_stable), (ustep[:,None])*B_stable))
                B_stable=ustep[:,None]*B_stable

                max_B_stable=B_stable.max()
                B_stable/=max_B_stable
                max_logB+= np.log(max_B_stable)
                v_hat_stable = B_stable.T.dot(one)
                Bs_stable = v_hat_stable.sum()

                x_interm_t=x_t.copy()
                x_interm_t[:n]+=np.log(ustep)
                                
                primal_var_t = B_stable/Bs_stable
                
                grad_psi_x_t = np.zeros_like(x_t)
                grad_psi_x_t[n:] = gamma*(-q + v_hat_stable/Bs_stable)
                psi_x_t = gamma*(np.log(Bs_stable.sum())+max_logB-x_interm_t[:n].dot(p)-x_interm_t[n:].dot(q))
                
                with warnings.catch_warnings():
                    warnings.filterwarnings('error')
                    try:
                        vstep = q/v_hat_stable
                    except Warning as e:
                        v_hat_stable/=v_hat_stable.max()
                        v_hat_stable[v_hat_stable<1e-150] = 1e-150
                        vstep = q/v_hat_stable
                vstep/=vstep.max()
                #print('veq', np.array_equal(B_stable.dot(np.diag(vstep)), B_stable*(vstep[None,:])))
                B_stable=B_stable*vstep[None,:]
                
                
                max_B_stable=B_stable.max()
                B_stable/=max_B_stable
                max_logB+= np.log(max_B_stable)

                y_t = x_interm_t.copy()
                y_t[n:] += np.log(vstep)
                psi_y_t = gamma*(np.log(B_stable.sum())+max_logB-y_t[:n].dot(p)-y_t[n:].dot(q))

                z_t = z - (a_t/sigma) * grad_psi_x_t;
            else:
                with warnings.catch_warnings():
                    warnings.filterwarnings('error')
                    try:
                        vstep = q/v_hat_stable
                    except Warning as e:
                        v_hat_stable/=v_hat_stable.max()
                        v_hat_stable[v_hat_stable<1e-150] = 1e-150
                        vstep = q/v_hat_stable
                vstep/=vstep.max()
                #print('veq', np.array_equal(B_stable.dot(np.diag(vstep)), B_stable*(vstep[None,:])))
                B_stable=B_stable*vstep[None,:]
                max_B_stable=B_stable.max()
                B_stable/=max_B_stable
                max_logB+= np.log(max_B_stable)
                u_hat_stable = B_stable.dot(one)
                Bs_stable = u_hat_stable.sum()
                
  
                x_interm_t=x_t.copy()
                x_interm_t[n:]+=np.log(vstep)
                
                primal_var_t = B_stable/Bs_stable
                
                grad_psi_x_t = np.zeros_like(x_t)
                grad_psi_x_t[:n] = gamma*(-p + u_hat_stable/Bs_stable)
                psi_x_t = gamma*(np.log(B_stable.sum())+max_logB-x_interm_t[:n].dot(p)-x_interm_t[n:].dot(q))
                if False:
                    logB = (K + np.outer(x_interm_t[:n], one) + np.outer(one, x_interm_t[n:]))
                    max_logB1 =logB.max()
                    logB_stable = logB - max_logB1
                    B_stable1 = np.exp(logB_stable)
                    print('3diff', gamma*(np.log(B_stable1.sum())+max_logB1-x_interm_t[:n].dot(p)-x_interm_t[n:].dot(q))- psi_x_t)
                    print('3', (B_stable-  B_stable1).max())
                

                with warnings.catch_warnings():
                    warnings.filterwarnings('error')
                    try:
                        ustep = p/u_hat_stable
                    except Warning as e:
                        u_hat_stable/=u_hat_stable.max()
                        u_hat_stable[u_hat_stable<1e-150] = 1e-150
                        ustep = p/u_hat_stable
                ustep/=ustep.max()
                #print('ueq', np.array_equal(np.diag(ustep).dot(B_stable), (ustep[:,None])*B_stable))
                B_stable=ustep[:,None]*B_stable

                max_B_stable=B_stable.max()
                B_stable/=max_B_stable
                max_logB += np.log(max_B_stable)


                y_t=x_interm_t.copy()
                y_t[:n]+=np.log(ustep)
                
                psi_y_t = gamma*(np.log(B_stable.sum())+max_logB-y_t[:n].dot(p)-y_t[n:].dot(q))

                B_stable1 = B_stable.copy()
                
                z_t = z - (a_t/sigma) * grad_psi_x_t; #trial of z_k
            
            upper = A_t*psi_y_t-A*psi_y;
            lower = a_t*psi_x_t+a_t*grad_psi_x_t.dot(z_t-x_interm_t)+((a_t**2)/(2*sigma))*((grad_psi_x_t*grad_psi_x_t).sum());
            l = upper-lower;

            if l <= 0:
                flag = 0; #end of the inner cycle flag
                x = x_t.copy(); #set x_{k+1}
                y = y_t.copy(); #set y_{k+1}
                z = z_t.copy(); #set y_{k+1}
                psi_y = psi_y_t; #save psi(y_k)
                A = A_t; #set A_{k+1}
                a=a_t;
                primal_var = tau * primal_var_t + (1-tau) * primal_var;
            j = j + 1;
        
        k-=-1
        
        if (C * (B_round(primal_var) - primal_var)).sum() <= eps/6 and abs(f_primal(primal_var) + psi_y_t) <= eps/6:
                return time.perf_counter() - start_time
        
        

In [0]:
def sinkhorn(x0, min_iter=0, min_time=0):
    u = x0[:n].copy()
    v = x0[n:].copy()
    vstep=np.zeros_like(v)
    ustep=np.zeros_like(u)
    start_time = time.perf_counter()
    k=0
    logB = (K + np.outer(u, one) + np.outer(one, v))
    max_logB =logB.max()
    logB_stable = logB - max_logB
    
    while True:
        B_stable = np.exp(logB_stable)    
        u_hat_stable = B_stable.sum(1)
        if k%10==0:
            Bs_stable = u_hat_stable.sum()
            
            #print(abs(v_hat_stable/Bs_stable - q).sum())
            if abs(u_hat_stable/Bs_stable - p).sum()  <= epsp/2:
                return time.perf_counter() - start_time
        
        
        ##first block
        with warnings.catch_warnings():
            warnings.filterwarnings('error')
            try:
                ustep = p/u_hat_stable
            except Warning as e:
                u_hat_stable/=u_hat_stable.max()
                u_hat_stable[u_hat_stable<1e-150] = 1e-150
                ustep = p/u_hat_stable
        ustep/=ustep.max()
        #u+=np.log(ustep)
        
        #print('eq', np.array_equal(logB_stable+np.log(ustep)[:,None] , logB_stable+np.outer(np.log(ustep), one)))
        logB_stable += np.log(ustep)[:,None]#np.outer(np.log(ustep), one)
        logB_stable-=logB_stable.max()

        B_stable = np.exp(logB_stable)    
        v_hat_stable = B_stable.sum(0)
        
        ##second block
        with warnings.catch_warnings():
            warnings.filterwarnings('error')
            try:
                vstep = q/v_hat_stable
            except Warning as e:
                v_hat_stable/=v_hat_stable.max()
                v_hat_stable[v_hat_stable<1e-150] = 1e-150
                vstep = q/v_hat_stable
        vstep/=vstep.max()

        logB_stable += np.log(vstep)[None,:]#np.outer(np.log(ustep), one)
        logB_stable-=logB_stable.max()
        
        k-=-1

In [0]:
#code is based on https://github.com/PythonOT/POT
def greenkhorn(x0):
    a = np.asarray(p, dtype=np.float64)
    b = np.asarray(q, dtype=np.float64)
    
    dim_a = a.shape[0]
    dim_b = b.shape[0]

    
    u = x0[:n].copy()
    v = x0[n:].copy()
    u = np.full(dim_a, 1. / dim_a)
    v = np.full(dim_b, 1. / dim_b)
    G = u[:, np.newaxis] * K * v[np.newaxis, :]

    viol = G.sum(1)## - a
    viol_2 = G.sum(0)## - b

    i=-1
    start_time = time.perf_counter()
    while True:
        i-=-1
        i_1 = np.argmax(np.abs(viol-a))
        i_2 = np.argmax(np.abs(viol_2-b))
        m_viol_1 = np.abs(viol[i_1]-a[i_1])
        m_viol_2 = np.abs(viol_2[i_2]-b[i_2])
        stopThr_val = np.maximum(m_viol_1, m_viol_2)

        if m_viol_1 > m_viol_2:
            old_u = u[i_1]
            u[i_1] = a[i_1] / (K[i_1, :].dot(v))
            G[i_1, :] = u[i_1] * K[i_1, :] * v

            viol[i_1] = u[i_1] * K[i_1, :].dot(v)## - a[i_1]
            viol_2 += (K[i_1, :].T * (u[i_1] - old_u) * v)

        else:
            old_v = v[i_2]
            v[i_2] = b[i_2] / (K[:, i_2].T.dot(u))
            G[:, i_2] = u * K[:, i_2] * v[i_2]
            #aviol = (G.sum(1) - a)
            #aviol_2 = (G.sum(0) - b)
            viol += (-old_v + v[i_2]) * K[:, i_2] * u
            viol_2[i_2] = v[i_2] * K[:, i_2].dot(u)## - b[i_2]

            #print(abs(viol/viol.sum() - a).sum() + abs(viol_2/viol_2.sum() - b).sum())

            if abs(viol/viol.sum() - a).sum() + abs(viol_2/viol_2.sum() - b).sum() <= epsp/2:
                сс=time.perf_counter() - start_time
                B_stable = G
            
                u_hat_stable, v_hat_stable = B_stable.dot(one), B_stable.T.dot(one)
                Bs_stable = u_hat_stable.sum()
                
                print(abs(viol/viol.sum() - a).sum() + abs(viol_2/viol_2.sum() - b).sum(), ' vs ', abs(u_hat_stable/Bs_stable - p).sum() + abs(v_hat_stable/Bs_stable - q).sum() )
                return сс

In [0]:
#experiments were done for
p_list = [34860, 31226,   239, 37372, 17390]
q_list = [45815, 35817, 43981, 54698, 49947,]

In [0]:
x0=np.zeros(2*n, np.float64)

In [0]:
x_array = np.linspace(1/2e-2, 1/4e-4, 6)
epslist = 1/x_array

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os.path

k=0
for k in range(len(p_list)):
    y_array = np.zeros_like(epslist)
    
    if not os.path.isfile('/content/drive/My Drive/colab/'+'aar_'+str(k)+'.npy'):
        i=0
        for eps in epslist:
            epsp = eps/8
            p,q = mnist(epsp, p_list[k], q_list[k])
            p_ref, q_ref = mnist(0, p_list[k], q_list[k])
            gamma=eps/3/np.log(n)
            K=-C/gamma
            y_array[i] = AARBCD(x0)
            i-=-1
        np.save('/content/drive/My Drive/colab/'+'aar_'+str(k)+'.npy', y_array)
    print('aar_', k, ' - done', y_array)

    if not os.path.isfile('/content/drive/My Drive/colab/'+'apd_'+str(k)+'.npy'):
        i=0
        for eps in epslist:
            epsp = eps/8
            p,q = mnist(epsp, p_list[k], q_list[k])
            p_ref, q_ref = mnist(0, p_list[k], q_list[k])
            gamma=eps/3/np.log(n)
            K=-C/gamma
            y_array[i] = apd(x0)
            i-=-1
        np.save('/content/drive/My Drive/colab/'+'apd_'+str(k)+'.npy', y_array)
    print('apd_', k, ' - done', y_array)

    if not os.path.isfile('/content/drive/My Drive/colab/'+'agm_'+str(k)+'.npy'):
        i=0
        for eps in epslist:
            epsp = eps/8
            p,q = mnist(epsp, p_list[k], q_list[k])
            p_ref, q_ref = mnist(0, p_list[k], q_list[k])
            gamma=eps/3/np.log(n)
            K=-C/gamma
            y_array[i]=AGMsDR(x0)
            i-=-1
        np.save('/content/drive/My Drive/colab/'+'agm_'+str(k)+'.npy', y_array)
    print('agm_', k, ' - done', y_array)

    if not os.path.isfile('/content/drive/My Drive/colab/'+'aam_'+str(k)+'.npy'):
        i=0
        for eps in epslist:
            epsp = eps/8
            p,q = mnist(epsp, p_list[k], q_list[k])
            p_ref, q_ref = mnist(0, p_list[k], q_list[k])
            gamma=eps/3/np.log(n)
            K=-C/gamma
            y_array[i] = aam(x0)
            i-=-1
        np.save('/content/drive/My Drive/colab/'+'aam_'+str(k)+'.npy', y_array)
    print('aam_', k, ' - done', y_array)

    if not os.path.isfile('/content/drive/My Drive/colab/'+'sin_'+str(k)+'.npy'):  
        i=0
        for eps in epslist:
            epsp = eps/8
            p,q = mnist(epsp, p_list[k], q_list[k])
            p_ref, q_ref = mnist(0, p_list[k], q_list[k])
            gamma=eps/4/np.log(n)
            K=-C/gamma
            y_array[i]=sinkhorn(x0)
            i-=-1
        np.save('/content/drive/My Drive/colab/'+'sin_'+str(k)+'.npy', y_array)
    print('sin_', k, ' - done', y_array)

    if not os.path.isfile('/content/drive/My Drive/colab/'+'green_'+str(k)+'.npy'):    
        i=0
        for eps in epslist:
            epsp = eps/8
            p,q = mnist(epsp, p_list[k], q_list[k])
            p_ref, q_ref = mnist(0, p_list[k], q_list[k])
            gamma=eps/4/np.log(n)
            K=-C/gamma
            y_array[i]=greenkhorn(x0)
            i-=-1
        np.save('/content/drive/My Drive/colab/'+'green_'+str(k)+'.npy', y_array)
    print('green_', k, ' - done', y_array)


KeyboardInterrupt: ignored